# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'week52change': 1.22571,
 'week52high': 467.1,
 'week52low': 205.75,
 'marketcap': 1943532617727,
 'employees': 139018,
 'day200MovingAvg': 308.74,
 'day50MovingAvg': 382,
 'float': 4403170820,
 'avg10Volume': 46811359.7,
 'avg30Volume': 39196867.7,
 'ttmEPS': 13.5212,
 'ttmDividendRate': 3.19,
 'companyName': 'Apple, Inc.',
 'sharesOutstanding': 4435965334,
 'maxChangePercent': 442.4971,
 'year5ChangePercent': 2.8601,
 'year2ChangePercent': 1.171,
 'year1ChangePercent': 1.239,
 'ytdChangePercent': 0.522,
 'month6ChangePercent': 0.4001,
 'month3ChangePercent': 0.4791,
 'month1ChangePercent': 0.1958,
 'day30ChangePercent': 0.2693,
 'day5ChangePercent': 0.02,
 'nextDividendDate': '2020-08-20',
 'dividendYield': 0.007295934715952886,
 'nextEarningsDate': '2020-10-25',
 'exDividendDate': '2020-08-03',
 'peRatio': 35.36,
 'beta': 1.135442067403298}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

1.239

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,101.530,0.442000,N/A
1,AAL,14.320,-0.556900,N/A
2,AAP,157.953,0.057500,N/A
3,AAPL,453.090,1.280000,N/A
4,ABBV,97.010,0.446500,N/A
...,...,...,...,...
500,YUM,94.890,-0.226303,N/A
501,ZBH,139.200,0.003268,N/A
502,ZBRA,291.849,0.338773,N/A
503,ZION,35.040,-0.207068,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DXCM,443.050,1.948890,N/A
1,NVDA,457.930,1.911291,N/A
2,AMD,86.720,1.518418,N/A
3,CARR,29.450,1.428916,N/A
4,AAPL,453.090,1.280000,N/A
5,REGN,617.430,1.019492,N/A
6,WST,274.430,0.942171,N/A
7,PYPL,197.380,0.875216,N/A
8,LRCX,382.800,0.871007,N/A
9,QRVO,136.050,0.852102,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since 

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:1000000
1000000


In [9]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DXCM,443.050,1.948890,44
1,NVDA,457.930,1.911291,42
2,AMD,86.720,1.518418,226
3,CARR,29.450,1.428916,665
4,AAPL,453.090,1.280000,43
5,REGN,617.430,1.019492,31
6,WST,274.430,0.942171,71
7,PYPL,197.380,0.875216,99
8,LRCX,382.800,0.871007,51
9,QRVO,136.050,0.852102,144


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [11]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0        0.89505
1      0.0277228
2       0.546535
3       0.992079
4       0.889109
         ...    
500     0.221782
501     0.473267
502     0.845545
503     0.253465
504      0.79604
Name: One-Year Return Percentile, Length: 505, dtype: object
0       0.867327
1      0.0178218
2       0.819802
3       0.956436
4       0.643564
         ...    
500     0.421782
501     0.411881
502     0.817822
503     0.227723
504     0.786139
Name: Six-Month Return Percentile, Length: 505, dtype: object
0       0.79604
1      0.930693
2      0.835644
3      0.970297
4      0.449505
         ...   
500    0.289109
501    0.546535
502    0.635644
503    0.429703
504    0.813861
Name: Three-Month Return Percentile, Length: 505, dtype: object
0       0.685149
1       0.653465
2       0.748515
3       0.944554
4      0.0415842
         ...    
500     0.378218
501     0.908911
502     0.687129
503     0.481188
504     0.855446
Name: One-Month Return Percentile, Length: 505, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,97.860,N/A,0.456400,0.89505,0.184000,0.867327,0.254400,0.79604,0.100800,0.685149,N/A
1,AAL,13.879,N/A,-0.552700,0.0277228,-0.561000,0.0178218,0.379000,0.930693,0.095000,0.653465,N/A
2,AAP,153.199,N/A,0.057100,0.546535,0.144500,0.819802,0.283100,0.835644,0.116800,0.748515,N/A
3,AAPL,451.552,N/A,1.256000,0.992079,0.393600,0.956436,0.468300,0.970297,0.197400,0.944554,N/A
4,ABBV,96.690,N/A,0.443200,0.889109,0.007100,0.643564,0.106300,0.449505,-0.066300,0.0415842,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,92.140,N/A,-0.232215,0.221782,-0.117754,0.421782,0.055747,0.289109,0.050721,0.378218,N/A
501,ZBH,140.090,N/A,0.003279,0.473267,-0.123155,0.411881,0.143863,0.546535,0.177478,0.908911,N/A
502,ZBRA,290.574,N/A,0.348686,0.845545,0.142139,0.817822,0.170794,0.635644,0.101048,0.687129,N/A
503,ZION,35.900,N/A,-0.205256,0.253465,-0.267590,0.227723,0.101673,0.429703,0.067879,0.481188,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [12]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [13]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
portfolio_input()

Enter the value of your portfolio:1000000


In [15]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,97.860,200,0.456400,0.89505,0.184000,0.867327,0.254400,0.79604,0.100800,0.685149,0.810891
1,AAL,13.879,1412,-0.552700,0.0277228,-0.561000,0.0178218,0.379000,0.930693,0.095000,0.653465,0.407426
2,AAP,153.199,127,0.057100,0.546535,0.144500,0.819802,0.283100,0.835644,0.116800,0.748515,0.737624
3,AAPL,451.552,43,1.256000,0.992079,0.393600,0.956436,0.468300,0.970297,0.197400,0.944554,0.965842
4,ABBV,96.690,202,0.443200,0.889109,0.007100,0.643564,0.106300,0.449505,-0.066300,0.0415842,0.505941
5,ABC,106.980,183,0.193200,0.69901,0.111100,0.778218,0.155900,0.582178,0.040100,0.336634,0.59901
6,ABMD,310.231,63,0.644500,0.966337,0.673700,0.992079,0.660500,0.994059,0.178200,0.914851,0.966832
7,ABT,102.280,191,0.206000,0.722772,0.152000,0.827723,0.080000,0.366337,0.095300,0.657426,0.643564
8,ACN,239.870,81,0.233000,0.738614,0.094800,0.760396,0.245200,0.782178,0.071200,0.50099,0.695545
9,ADBE,449.720,43,0.581100,0.944554,0.234000,0.89901,0.227900,0.742574,0.000500,0.186139,0.693069


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that ormats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [19]:
writer.save()